In [2]:
from langame import LangameClient
from pprint import pprint
from firebase_admin import firestore
import requests
from bs4 import BeautifulSoup
import json
import openai
from pathlib import Path
import pandas as pd
import datetime
import markdown
c = LangameClient(path_to_config_file="../config.yaml")

In [17]:
!rm -rf $out_file_name

In [18]:
out_file_name = "../data/openai_fine_tune_ice_breaker_26_11_2021.jsonl"
in_file_name = "../data/augment/ice_breaker_26_11_2021.jsonl"

out_file_name = "../data/openai_fine_tune_science_26_11_2021.jsonl"
in_file_name = "../data/augment/science_26_11_2021.jsonl"

with open(in_file_name) as f:
  for e in f.readlines():
    e_json = json.loads(e)
    if "topics" not in e_json or \
      "content" not in e_json or \
      not all(isinstance(x, str) for x in e_json["topics"]):
      continue
    topics = ",".join(e_json["topics"])
    content = e_json["content"]
    with open(out_file_name, "a+") as outfile:
      json.dump({
          "prompt": f"",
          "completion": f" {topics} ### {content}",
      }, outfile)
      outfile.write('\n')

In [9]:
!openai tools fine_tunes.prepare_data -f $out_file_name

Analyzing...

- Your file contains 8014 prompt-completion pairs
- There are 944 duplicated prompt-completion pairs. These are rows: [70, 75, 145, 190, 231, 287, 295, 334, 457, 466, 481, 499, 501, 514, 523, 568, 583, 617, 621, 649, 654, 689, 697, 706, 714, 745, 747, 754, 764, 812, 818, 839, 859, 862, 864, 895, 924, 928, 931, 942, 1011, 1015, 1036, 1038, 1060, 1068, 1070, 1106, 1119, 1178, 1179, 1187, 1190, 1204, 1242, 1294, 1302, 1310, 1322, 1338, 1358, 1375, 1391, 1392, 1397, 1404, 1409, 1416, 1433, 1439, 1443, 1448, 1453, 1461, 1464, 1471, 1472, 1500, 1504, 1506, 1525, 1563, 1574, 1575, 1587, 1610, 1613, 1616, 1618, 1631, 1657, 1663, 1674, 1700, 1701, 1712, 1717, 1721, 1742, 1753, 1767, 1772, 1776, 1784, 1786, 1795, 1804, 1807, 1823, 1824, 1832, 1838, 1851, 1863, 1875, 1890, 1894, 1902, 1904, 1907, 1916, 1925, 1926, 1932, 1980, 1981, 1995, 1996, 2012, 2016, 2022, 2026, 2040, 2050, 2053, 2057, 2068, 2091, 2093, 2110, 2112, 2121, 2136, 2142, 2144, 2156, 2183, 2206, 2210, 2215, 2235, 223

In [19]:
f = openai.File.create(
  file=open(out_file_name),
  purpose="fine-tune"
)
f_t = openai.FineTune.create(
    training_file=f["id"],
    model="curie"
)

In [24]:
model = openai.FineTune.list()['data'][-1]
model

<FineTune fine-tune id=ft-TjGJ42Q5BOnwKE6UZrNnJbli at 0x7f6770779e00> JSON: {
  "created_at": 1637945578,
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.05,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1,
    "use_packing": null
  },
  "id": "ft-TjGJ42Q5BOnwKE6UZrNnJbli",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-KwcHNgfGe4pqdKDLQIJt99UZ",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 3253504,
      "created_at": 1637945577,
      "filename": "openai_fine_tune_science_26_11_2021.jsonl",
      "id": "file-BaEQPIuY09261XNwuD73flUR",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1637945578,
  "validation_files": []
}

In [34]:
openai.Completion.create(
  model=model["fine_tuned_model"],
  prompt="",
  max_tokens=300,
  stop=["\n###\n", "###"],
)

<OpenAIObject text_completion id=cmpl-3bb9PbAkCra9NSEJrOTKDhDxi4yCP at 0x13f15c770> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "While most of humanity is focused on avoiding harm, a small but very vocal minority has decided that the best way to avoid harm is to actively seeks it out. Called \u201cthe optimists\u201d by the media, this group includes anarchists, Daredevil bloggers, thrill-seekers, and other \u2013 sometimes bizarre \u2013 personalities."
    }
  ],
  "created": 1630089955,
  "id": "cmpl-3bb9PbAkCra9NSEJrOTKDhDxi4yCP",
  "model": "curie:ft-louis-personal-2021-08-27-16-57-14",
  "object": "text_completion"
}